# Lab5: Machine Translation
# Yucheng Jin (yucheng9)

Machine translation is an application of NLP to computationally translate text from one language to the other. It is one of the most popular fields of research in the NLP community. Very recently (2014) deep learning methods changed the face of machine translation and all of NLP. The application of deep learning to machine translation was referred to as Neural Machine Translation (NMT).

In this mini lab, we will see a sample code that performs neural machine translation, and we will see the effects of various hyperparameters on the performance of the system.

For simplicity, we will take as our dataset, a set of short English sentences mapped to French sentences, and instead of performing the translation one word at a time, it will be performing it one character at a time.

Total points: 20 points + 10 bonus points

**Submission Instructions**: Just upload this notebook, with all your answers in the respective cells, to Compass.

The following block of code contains the full code of the functional NMT system. Run it to get a sense of how the loss and accuracy of the training data and the validation data are changing with every epoch. No data is necessary to download; all is included in the lab's directory.

#### Packages to install:
- pip install keras
- pip install tensorflow

In [2]:
'''
#Sequence to sequence example in Keras (character-level).
This script demonstrates how to implement a basic character-level
sequence-to-sequence model. We apply it to translating
short English sentences into short French sentences,
character-by-character. Note that it is fairly unusual to
do character-level machine translation, as word-level
models are more common in this domain.
**Summary of the algorithm**
- We start with input sequences from a domain (e.g. English sentences)
    and corresponding target sequences from another domain
    (e.g. French sentences).
- An encoder LSTM turns input sequences to 2 state vectors
    (we keep the last LSTM state and discard the outputs).
- A decoder LSTM is trained to turn the target sequences into
    the same sequence but offset by one timestep in the future,
    a training process called "teacher forcing" in this context.
    It uses as initial state the state vectors from the encoder.
    Effectively, the decoder learns to generate `targets[t+1...]`
    given `targets[...t]`, conditioned on the input sequence.
- In inference mode, when we want to decode unknown input sequences, we:
    - Encode the input sequence into state vectors
    - Start with a target sequence of size 1
        (just the start-of-sequence character)
    - Feed the state vectors and 1-char target sequence
        to the decoder to produce predictions for the next character
    - Sample the next character using these predictions
        (we simply use argmax).
    - Append the sampled character to the target sequence
    - Repeat until we generate the end-of-sequence character or we
        hit the character limit.
**Data download**
[English to French sentence pairs.
](http://www.manythings.org/anki/fra-eng.zip)
'''
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64  # Batch size for training.
epochs = 10  # Number of epochs to train for.
latent_dim = 100  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'fra-eng/fra.txt'

# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')

# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

Using TensorFlow backend.


Number of samples: 10000
Number of unique input tokens: 70
Number of unique output tokens: 93
Max sequence length for inputs: 16
Max sequence length for outputs: 59
Train on 8000 samples, validate on 2000 samples
Epoch 1/10
8000/8000 [==============================] - 10s 1ms/step - loss: 1.2368 - accuracy: 0.7221 - val_loss: 1.1127 - val_accuracy: 0.6966
Epoch 2/10
8000/8000 [==============================] - 9s 1ms/step - loss: 0.9124 - accuracy: 0.7481 - val_loss: 0.9669 - val_accuracy: 0.7341
Epoch 3/10
8000/8000 [==============================] - 8s 1ms/step - loss: 0.7940 - accuracy: 0.7838 - val_loss: 0.8391 - val_accuracy: 0.7688
Epoch 4/10
8000/8000 [==============================] - 9s 1ms/step - loss: 0.6874 - accuracy: 0.8065 - val_loss: 0.7541 - val_accuracy: 0.7847
Epoch 5/10
8000/8000 [==============================] - 8s 996us/step - loss: 0.6226 - accuracy: 0.8210 - val_loss: 0.6956 - val_accuracy: 0.8009
Epoch 6/10
8000/8000 [==============================] - 8s 1ms/s

When training an NMT system (or any deep learning system), one of the decisions to take is the batch size: number of examples processed (in parallel) before updating parameters.

**Deliverable 1** (5 points): Change the batch size from 64 to 128, and then 256. How did it affect the speed of training (time per epoch)? Is it slower or faster? Why? What was its effect on the increase of accuracy from one epoch to the other? What is the concluded tradeoff between a smaller batch size and a larger one?

In [15]:
# data here, code below
import statistics
from statistics import mean
import pandas as pd
b_64 = [9, 9, 9, 10, 9, 8, 9, 8, 8, 8]
b_128 = [8, 9, 8, 9, 9, 9, 9, 8, 8, 8]
b_256 = [8, 8, 8, 8, 8, 8, 8, 8, 8, 8]
options_list = ["batch size = 64", "batch size = 128", "batch size = 256"]
b_64.append(mean(b_64))
b_128.append(mean(b_128))
b_256.append(mean(b_256))
data = [b_64, b_128, b_256] 
title = list(range(1,11))
title.append('average time (s)')
df = pd.DataFrame(data, options_list, title)
df.name = "Time Per Epoch of Different Batch Sizes"

b_64_a = [0.7229, 0.7459, 0.7864, 0.8098, 0.8231, 0.8325, 0.8401, 0.8461, 0.8507, 0.8549, 0.8549-0.7229]
b_128_a = [0.7139, 0.7333, 0.7413, 0.7595, 0.7832, 0.7989, 0.8092, 0.8177, 0.8240, 0.8304, 0.8304-0.7139]
b_256_a = [0.6949, 0.7271, 0.7331, 0.7341, 0.7390, 0.7463, 0.7579, 0.7702, 0.7833, 0.7915, 0.7915-0.6949]
options_list = ["batch size = 64", "batch size = 128", "batch size = 256"]
b_64_a.append(mean(b_64_a))
b_128_a.append(mean(b_128_a))
b_256_a.append(mean(b_256_a))
data_2 = [b_64_a, b_128_a, b_256_a] 
title_2 = list(range(1,11))
title_2.append('increase of accuracy')
title_2.append('average accuracy')
df_2 = pd.DataFrame(data_2, options_list, title_2)
df_2.name = "Accuracy of Every Epoch of Different Batch Sizes"

print(df.name)
df

Time Per Epoch of Different Batch Sizes


,1,2,3,4,5,6,7,8,9,10,average time (s)
batch size = 64,9,9,9,10,9,8,9,8,8,8,8.7
batch size = 128,8,9,8,9,9,9,9,8,8,8,8.5
batch size = 256,8,8,8,8,8,8,8,8,8,8,8.0


In [16]:
print(df_2.name)
df_2

Accuracy of Every Epoch of Different Batch Sizes


,1,2,3,4,5,6,7,8,9,10,increase of accuracy,average accuracy
batch size = 64,0.7229,0.7459,0.7864,0.8098,0.8231,0.8325,0.8401,0.8461,0.8507,0.8549,0.1320,0.749491
batch size = 128,0.7139,0.7333,0.7413,0.7595,0.7832,0.7989,0.8092,0.8177,0.8240,0.8304,0.1165,0.720718
batch size = 256,0.6949,0.7271,0.7331,0.7341,0.7390,0.7463,0.7579,0.7702,0.7833,0.7915,0.0966,0.688545


**My Answer**

Another decision to make about the architecture of the NMT is the latent dimension size of the LSTM block. The LSTM block encodes semantics of a sentence to a vector of size equal to the latent dimension. This controls the modeling capacity of our system.

**Deliverable 2** (5 points): Change the latent dimension from a 100 to a 10, and a 1000. How did it affect speed of training? Why? How did affect the accuracy of the system?

In [28]:
l_100 = [9, 9, 9, 10, 9, 8, 9, 8, 8, 8]
l_10 = [5, 4, 3, 4, 4, 4, 3, 3, 3, 3]
l_1000 = [206, 209, 206, 210, 219, 207, 202, 211, 203, 210]
options_list = ["latent dimension = 10", "latent dimension = 100", "latent dimension = 1000"]
l_10.append(mean(l_10))
l_100.append(mean(l_100))
l_1000.append(mean(l_1000))
data = [l_10, l_100, l_1000] 
title = list(range(1,11))
title.append('average time (s)')
df = pd.DataFrame(data, options_list, title)
df.name = "Time Per Epoch of Different Latent Dimension"

l_100_a = [0.7229, 0.7459, 0.7864, 0.8098, 0.8231, 0.8325, 0.8401, 0.8461, 0.8507, 0.8549]
l_10_a = [0.6537, 0.7225, 0.7225, 0.7258, 0.7394, 0.7481, 0.7516, 0.7604, 0.7732, 0.7788]
l_1000_a = [0.7225, 0.7842, 0.8086, 0.8332, 0.8499, 0.8653, 0.8780, 0.8898, 0.8999, 0.9084]
options_list = ["latent dimension = 10", "latent dimension = 100", "latent dimension = 1000"]
l_10_a.append(mean(l_10_a))
l_100_a.append(mean(l_100_a))
l_1000_a.append(mean(l_1000_a))
data_2 = [l_10_a, l_100_a, l_1000_a] 
title_2 = list(range(1,11))
title_2.append('average accuracy')
df_2 = pd.DataFrame(data_2, options_list, title_2)
df_2.name = "Accuracy of Every Epoch of Different Batch Sizes"

print(df.name)
df

Time Per Epoch of Different Latent Dimension


,1,2,3,4,5,6,7,8,9,10,average time (s)
latent dimension = 10,5,4,3,4,4,4,3,3,3,3,3.6
latent dimension = 100,9,9,9,10,9,8,9,8,8,8,8.7
latent dimension = 1000,206,209,206,210,219,207,202,211,203,210,208.3


In [29]:
print(df_2.name)
df_2

Accuracy of Every Epoch of Different Batch Sizes


,1,2,3,4,5,6,7,8,9,10,average accuracy
latent dimension = 10,0.6537,0.7225,0.7225,0.7258,0.7394,0.7481,0.7516,0.7604,0.7732,0.7788,0.73760
latent dimension = 100,0.7229,0.7459,0.7864,0.8098,0.8231,0.8325,0.8401,0.8461,0.8507,0.8549,0.81124
latent dimension = 1000,0.7225,0.7842,0.8086,0.8332,0.8499,0.8653,0.8780,0.8898,0.8999,0.9084,0.84398


**My Answer**

**Deliverable 3** (5 points): One of the issues with the code above is that at demo time (for loop at the bottom of the code), the test is performed on training instances instead of validation instances. Adjust the code to perform the demo on the first 10 instances of the validation instances, and copy the cell's output below:

**My Answer**

**Deliverable 4** (5 points): In the following piece of code `decoder_dense = Dense(num_decoder_tokens, activation='softmax')` a dense layer is instantiated to map the output of the LSTM block to a prediction of the next character. Why is the output size of this dense layer set to `num_decoder_tokens`, which is the number of possible French output characters?

In [20]:
print(num_decoder_tokens)
print(len(target_characters))

93
93


**My Answer**

**Bonus Question** (10 points): During training, the model, besides being fed the target data (French translation) as in the normal supervised procedure, it is also fed the target data shifted by one position to the left. What is the purpose of this practice?  

**My Answer**

# Deliverable 1 Code

In [3]:
batch_size = 256  # Batch size for training, change from 64 to 128 to 256.
epochs = 10  # Number of epochs to train for.
latent_dim = 100  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'fra-eng/fra.txt'

# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')

# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

Number of samples: 10000
Number of unique input tokens: 70
Number of unique output tokens: 93
Max sequence length for inputs: 16
Max sequence length for outputs: 59
Train on 8000 samples, validate on 2000 samples
Epoch 1/10
8000/8000 [==============================] - 8s 1ms/step - loss: 1.7467 - accuracy: 0.6949 - val_loss: 1.2967 - val_accuracy: 0.6901
Epoch 2/10
8000/8000 [==============================] - 8s 981us/step - loss: 1.1033 - accuracy: 0.7271 - val_loss: 1.2333 - val_accuracy: 0.6959
Epoch 3/10
8000/8000 [==============================] - 8s 981us/step - loss: 1.0366 - accuracy: 0.7331 - val_loss: 1.1389 - val_accuracy: 0.6962
Epoch 4/10
8000/8000 [==============================] - 8s 976us/step - loss: 0.9847 - accuracy: 0.7341 - val_loss: 1.0803 - val_accuracy: 0.7005
Epoch 5/10
8000/8000 [==============================] - 8s 975us/step - loss: 0.9443 - accuracy: 0.7390 - val_loss: 1.0387 - val_accuracy: 0.6977
Epoch 6/10
8000/8000 [==============================] - 8s 

-
Input sentence: We won.
Decoded sentence: Tous e  ai                                                  
-
Input sentence: We won.
Decoded sentence: Tous e  ai                                                  
-
Input sentence: We won.
Decoded sentence: Tous e  ai                                                  
-
Input sentence: We won.
Decoded sentence: Tous e  ai                                                  
-
Input sentence: Ask Tom.
Decoded sentence: Arete                                                       
-
Input sentence: Awesome!
Decoded sentence: Ares e                                                      
-
Input sentence: Be calm.
Decoded sentence: Ales e                                                      
-
Input sentence: Be calm.
Decoded sentence: Ales e                                                      
-
Input sentence: Be calm.
Decoded sentence: Ales e                                                      
-
Input sentence: Be cool.
Decoded sentence: Ales 

# Deliverable 2 Code

In [25]:
batch_size = 64  # Batch size for training.
epochs = 10  # Number of epochs to train for.
latent_dim = 1000  # Latent dimensionality of the encoding space, change from 10 to 100 to 1000.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'fra-eng/fra.txt'

# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')

# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

Number of samples: 10000
Number of unique input tokens: 70
Number of unique output tokens: 93
Max sequence length for inputs: 16
Max sequence length for outputs: 59
Train on 8000 samples, validate on 2000 samples
Epoch 1/10
8000/8000 [==============================] - 206s 26ms/step - loss: 1.2202 - accuracy: 0.7225 - val_loss: 1.0714 - val_accuracy: 0.7065
Epoch 2/10
8000/8000 [==============================] - 209s 26ms/step - loss: 0.8203 - accuracy: 0.7842 - val_loss: 0.7749 - val_accuracy: 0.7784
Epoch 3/10
8000/8000 [==============================] - 206s 26ms/step - loss: 0.6708 - accuracy: 0.8086 - val_loss: 0.7021 - val_accuracy: 0.7971
Epoch 4/10
8000/8000 [==============================] - 210s 26ms/step - loss: 0.5693 - accuracy: 0.8332 - val_loss: 0.6212 - val_accuracy: 0.8163
Epoch 5/10
8000/8000 [==============================] - 219s 27ms/step - loss: 0.5075 - accuracy: 0.8499 - val_loss: 0.5704 - val_accuracy: 0.8311
Epoch 6/10
8000/8000 [==============================

# Deliverable 3 Code

In [6]:
batch_size = 256  # Batch size for training.
epochs = 10  # Number of epochs to train for.
latent_dim = 100  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'fra-eng/fra.txt'

# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')

# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

validation = int(len(encoder_input_data)*0.8)
for seq_index in range(10):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[validation + seq_index:validation + seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[validation + seq_index])
    print('Decoded sentence:', decoded_sentence)

Number of samples: 10000
Number of unique input tokens: 70
Number of unique output tokens: 93
Max sequence length for inputs: 16
Max sequence length for outputs: 59
Train on 8000 samples, validate on 2000 samples
Epoch 1/10
8000/8000 [==============================] - 10s 1ms/step - loss: 1.8094 - accuracy: 0.6979 - val_loss: 1.2700 - val_accuracy: 0.6929
Epoch 2/10
8000/8000 [==============================] - 8s 1ms/step - loss: 1.1038 - accuracy: 0.7295 - val_loss: 1.1775 - val_accuracy: 0.6959
Epoch 3/10
8000/8000 [==============================] - 8s 1ms/step - loss: 1.0395 - accuracy: 0.7328 - val_loss: 1.1190 - val_accuracy: 0.6959
Epoch 4/10
8000/8000 [==============================] - 8s 1ms/step - loss: 0.9848 - accuracy: 0.7333 - val_loss: 1.0745 - val_accuracy: 0.6999
Epoch 5/10
8000/8000 [==============================] - 9s 1ms/step - loss: 0.9423 - accuracy: 0.7368 - val_loss: 1.0491 - val_accuracy: 0.7030
Epoch 6/10
8000/8000 [==============================] - 10s 1ms/st